# Outline

- Simple hashing - hotel with 26 rooms, labeled A-Z. Guests are assigned to room based on their first name initial. If room occupied guest is not admitted.
- Not a good business model. Lots of demand for room `J` and `M` because John, Mary very popular names. Less demand for `X` or `Z`.
- Superfast to tell if a person is a guest at the hotel. If room corresponding to person's first name initial is not empty, person with that name is present. $\mathcal O(1)$.
- Brief/simple intro to $\mathcal O$ notation.
- Many room remain empty. Introduce probing. John goes to `J`. James cannot go to `J` because John is there but what if the next room `K` is available. James can go there. Now, it may take 2 steps to find if James is in hotel. We expect him in `J` but we'll also probe `K`. Still faster than linear search or binary search.
- Discuss probing.
- Probing requires that we take over rooms that should be available for other guests.
- How about moving roll-in beds to a room as more guest with the same name initial arrive at the hotel (ok, it will get a bit crowded, but we are not a 5-star hotel).
- Introduce simple module based hashing. Instead of a hotel with 26 rooms, for A, B, .., Z, use `ord(initial)%capacity` to determine room number.
- discuss and showcase how to rehash when the load factor exceeds some threshold. Define load factor first. Demonstrate the difference. Ensure reusability of code when writing the rehash function.
- In building a class (Hotel Alphabetical) discuss some standardized behavior (based on software contracts). For example, the objects must have an `exists -> bool` function. A `remove -> guest` etc. These two are ideal for an assignmemnt.

# Reading material besides the textbook

## Hash specific

- [Chapter 11: Hash Tables](https://learning.oreilly.com/library/view/data-structures/9780134855912/ch11.xhtml) from _Data Structures & Algorithms in Python_ by John Canning, Alan Broder, and Robert Lafore. Available from O'Reilly at no cost with LUC login.

- [Hans Peter Luhn and the Birth of the Hashing Algorithm](https://spectrum.ieee.org/hans-peter-luhn-and-the-birth-of-the-hashing-algorithm), IEEE Spectrum 2018.

## Python programming in general

- [Effective Python](https://learning.oreilly.com/library/view/effective-python-125/9780138172398/): an good book, similar to Bloch's _Effective Java_ with useful tips for solid programming. Available from O'Reilly at no cost with LUC login.

- Bill Lubanovic's [Introducing Python](https://learning.oreilly.com/library/view/introducing-python-3rd/9781098174392/) is an excellent resource in general. Available from O'Reilly at no cost with LUC login.

- [Python Cookbook](https://learning.oreilly.com/library/view/python-cookbook-3rd/9781449357337/) by Dave Beazley. A bit dated (2006) but current and relevant. Available from O'Reilly at no cost with LUC login.

- [Robust Python](https://learning.oreilly.com/library/view/robust-python/9781098100650/) by Patrick Viafore is a bit advanced but some good intro stuff can be used in the course. Available from O'Reilly at no cost with LUC login.

- [Fluent Python](https://learning.oreilly.com/library/view/fluent-python-2nd/9781492056348/) by Luciano Ramalho is my "go to" place for ABCs and patterns. Available from O'Reilly at no cost with LUC login.

# Summary of assignments

This assignment comprises 2 problems.

- Write the `exists` method
- Write the `delete` method


# Assignment

Given classes `Guest` and `HotelAlphabetical` below, design and implement the following methods.

- `HotelAlphabetical.exists(self, guest_name:str) -> bool:` that returns true if a guest with the specified name is present in the hotel and false otherwise.

- `HotelAlphabetical.remove(self, guest_name:str) -> "Guest" | None:` that removes a guest, if present, from the hotel. If the removal is successful, the guest object is returned; otherwise the method returns `None`.


In [ ]:
class Guest:
    """Class representing guests for our Hotel Alphabetical. These objects
    are essentially nodes in a linked list. Each hotel room is effectively a
    linked list of guests, placed in that room based on some hashing function.
    """

    def __init__(self, name: str):
        self.name = name
        self.next = None  # Pointer to the next guest in the linked list

    def __repr__(self):
        return f"Guest({self.name})"

    def set_next(self, next_guest):
        self.next = next_guest

    def get_next(self):
        return self.next

    def has_next(self):
        return self.next is not None

    def get_name(self):
        return self.name

    def __str__(self):
        return self.name()


class HotelAlphabetical:
    """Class representing a hotel where guests are stored in rooms based on
    the first initial of their names. Each room is a linked list of guests.
    """

    _DEFAULT_CAPACITY = 26
    _ASCII_LEFT_EDGE = ord("A")
    _ASCII_RIGHT_EDGE = ord("Z")
    _EMPTY = "boohoo, your hotel is empty."
    _NEXT_GUEST = " --> "

    _LOAD_FACTOR_THRESHOLD = 0.7
    _INCREMENT_FACTOR = 2

    def __init__(self, capacity: int = _DEFAULT_CAPACITY):
        self._capacity = capacity
        self._hotel = [None] * capacity  # Array of linked lists for each letter
        self._usage = 0  # number of array slots used
        self._size = 0

    def _get_index(self, name: str) -> int:
        """Compute the index in the hotel array based on the first
        initial of the guest's name."""
        # Default to 0 if name is None or empty or not A-Z
        room_index = 0
        if name is not None and len(name) > 0:
            # DISCUSSION POINT: should we be computing the first initial
            # here or should it be done in object Guest?
            initial_ascii = ord(name.upper()[0])
            if self._ASCII_LEFT_EDGE <= initial_ascii <= self._ASCII_RIGHT_EDGE:
                room_index = initial_ascii % self._capacity
        return room_index

    def _check_load_factor(self) -> bool:
        """Check if the load factor exceeds the threshold."""
        load_factor = self._usage / self._capacity
        return load_factor > self._LOAD_FACTOR_THRESHOLD

    def _rehash(self) -> None:
        """Rehash the hotel by increasing its capacity and reassigning guests."""
        # Preserve the old hotel array and its capacity
        old_hotel = self._hotel
        old_capacity = self._capacity
        # Create a new hotel array with increased capacity
        self._capacity *= self._INCREMENT_FACTOR
        # Initialize the new hotel array and reset usage
        self._hotel = [None] * self._capacity
        self._usage = 0
        # Reinsert all guests into the new hotel array
        for room in range(old_capacity):
            guest_in_room = old_hotel[room]
            while guest_in_room is not None:
                self.add_guest(guest_in_room.get_name())
                guest_in_room = guest_in_room.get_next()

    def add_guest(self, name: str) -> None:
        """Add a guest to the hotel."""
        if self._check_load_factor():
            self._rehash()
        # Compute the room index based on the first initial of the name
        room = self._get_index(name)
        # Create a new guest object
        guest = Guest(name)
        # Insert the guest at the front of the linked list for that room
        if self._hotel[room] is None:
            self._hotel[room] = guest
            self._usage += 1
        else:
            guest.set_next(self._hotel[room])
            self._hotel[room] = guest
        # Increment the current occupancy of the hotel
        self._size += 1

    def __repr__(self) -> str:
        hotel_string = self._EMPTY
        if self._size > 0:
            hotel_string = f"\nThere are {self._size} guest(s) in your hotel."
            hotel_string += f"\nThe hotel has a capacity of {self._capacity} rooms."
            hotel_string += f" and is using {self._usage} room(s)."
            hotel_string += f"\nThe load factor is {self._usage/self._capacity:.2f}."
            hotel_string += f" The {self._size} guest(s) are:"
            for room in range(self._capacity):
                if self._hotel[room] is not None:
                    hotel_string += f"\n\tRoom {room:02d}: "
                    guest_in_room = self._hotel[room]
                    while guest_in_room is not None:
                        hotel_string += f"{guest_in_room.get_name()}{self._NEXT_GUEST}"
                        guest_in_room = guest_in_room.get_next()
                    hotel_string += ""
        return hotel_string

In [ ]:
# DEMO
test = HotelAlphabetical(4)
# Obligatory statement about how bad idea is to write multi-statement lines
# but it is just for testing purposes here
test.add_guest("Alice")
test.add_guest("Aaron")
test.add_guest("Ava")
print(test)
test.add_guest("Bob")
test.add_guest("Charlie")
test.add_guest("David")
print(test)
test.add_guest("Eve")
test.add_guest("Zara")
test.add_guest("Yvonne")
test.add_guest("Xander")
test.add_guest("Mallory")
test.add_guest("Trent")
test.add_guest("Peggy")
test.add_guest("Victor")
test.add_guest("Walter")
print(test)


There are 3 guest(s) in your hotel.
The hotel has a capacity of 4 rooms. and is using 1 room(s).
The load factor is 0.25. The 3 guest(s) are:
	Room 01: Ava --> Aaron --> Alice --> 

There are 11 guest(s) in your hotel.
The hotel has a capacity of 8 rooms. and is using 4 room(s).
The load factor is 0.50. The 11 guest(s) are:
	Room 01: Alice --> Aaron --> Ava --> 
	Room 02: Bob --> 
	Room 03: Charlie --> 
	Room 04: David --> 

There are 30 guest(s) in your hotel.
The hotel has a capacity of 16 rooms. and is using 12 room(s).
The load factor is 0.75. The 30 guest(s) are:
	Room 00: Peggy --> 
	Room 01: Ava --> Aaron --> Alice --> 
	Room 02: Bob --> 
	Room 03: Charlie --> 
	Room 04: Trent --> David --> 
	Room 05: Eve --> 
	Room 06: Victor --> 
	Room 07: Walter --> 
	Room 08: Xander --> 
	Room 09: Yvonne --> 
	Room 10: Zara --> 
	Room 13: Mallory --> 


---
# SOLUTIONS
---


In [21]:
class HotelAlphabetical_SOLUTION:
    """Class representing a hotel where guests are stored in rooms based on
    the first initial of their names. Each room is a linked list of guests.
    """

    _DEFAULT_CAPACITY = 26
    _ASCII_LEFT_EDGE = ord("A")
    _ASCII_RIGHT_EDGE = ord("Z")
    _EMPTY = "boohoo, your hotel is empty."
    _NEXT_GUEST = " --> "

    _LOAD_FACTOR_THRESHOLD = 0.7
    _INCREMENT_FACTOR = 2

    def __init__(self, capacity: int = _DEFAULT_CAPACITY):
        self._capacity = capacity
        self._hotel = [None] * capacity  # Array of linked lists for each letter
        self._usage = 0  # number of array slots used
        self._size = 0

    def _get_index(self, name: str) -> int:
        """Compute the index in the hotel array based on the first
        initial of the guest's name."""
        # Default to 0 if name is None or empty or not A-Z
        room_index = 0
        if name is not None and len(name) > 0:
            # DISCUSSION POINT: should we be computing the first initial
            # here or should it be done in object Guest?
            initial_ascii = ord(name.upper()[0])
            if self._ASCII_LEFT_EDGE <= initial_ascii <= self._ASCII_RIGHT_EDGE:
                room_index = initial_ascii % self._capacity
        return room_index

    def _check_load_factor(self) -> bool:
        """Check if the load factor exceeds the threshold."""
        load_factor = self._usage / self._capacity
        return load_factor > self._LOAD_FACTOR_THRESHOLD

    def _rehash(self) -> None:
        """Rehash the hotel by increasing its capacity and reassigning guests."""
        # Preserve the old hotel array and its capacity
        old_hotel = self._hotel
        old_capacity = self._capacity
        # Create a new hotel array with increased capacity
        self._capacity *= self._INCREMENT_FACTOR
        # Initialize the new hotel array and reset usage
        self._hotel = [None] * self._capacity
        self._usage = 0
        # Reinsert all guests into the new hotel array
        for room in range(old_capacity):
            guest_in_room = old_hotel[room]
            while guest_in_room is not None:
                self.add_guest(guest_in_room.get_name())
                guest_in_room = guest_in_room.get_next()

    def add_guest(self, name: str) -> None:
        """Add a guest to the hotel."""
        if self._check_load_factor():
            self._rehash()
        # Compute the room index based on the first initial of the name
        room = self._get_index(name)
        # Create a new guest object
        guest = Guest(name)
        # Insert the guest at the front of the linked list for that room
        if self._hotel[room] is None:
            self._hotel[room] = guest
            self._usage += 1
        else:
            guest.set_next(self._hotel[room])
            self._hotel[room] = guest
        # Increment the current occupancy of the hotel
        self._size += 1

    def exists(self, name) -> bool:
        """Check if a guest with the given name exists in the hotel."""
        found = False
        # Determine the expected room based on the first initial of the name
        expected_room = self._get_index(name)
        # Traverse the linked list in that room to find the guest
        guest_in_room = self._hotel[expected_room]
        while guest_in_room is not None and not found:
            found = guest_in_room.get_name() == name
            guest_in_room = guest_in_room.get_next()
        return found

    def remove(self, name) -> Guest | None:
        """Remove a guest with the given name from the hotel."""
        removed_guest = None
        # Determine the expected room based on the first initial of the name
        expected_room = self._get_index(name)
        # Traverse the linked list in that room to find and remove the guest
        current_guest = self._hotel[expected_room]
        previous_guest = None
        while current_guest is not None and removed_guest is None:
            if current_guest.get_name() == name:
                removed_guest = current_guest
                if previous_guest is None:
                    # Removing the first guest in the room
                    self._hotel[expected_room] = current_guest.get_next()
                else:
                    # Bypass the current guest
                    previous_guest.set_next(current_guest.get_next())
                self._size -= 1
            else:
                previous_guest = current_guest
                current_guest = current_guest.get_next()
        return removed_guest

    def __repr__(self) -> str:
        hotel_string = self._EMPTY
        if self._size > 0:
            hotel_string = f"\nThere are {self._size} guest(s) in your hotel."
            hotel_string += f"\nThe hotel has a capacity of {self._capacity} rooms."
            hotel_string += f" and is using {self._usage} room(s)."
            hotel_string += f"\nThe load factor is {self._usage/self._capacity:.2f}."
            hotel_string += f" The {self._size} guest(s) are:"
            for room in range(self._capacity):
                if self._hotel[room] is not None:
                    hotel_string += f"\n\tRoom {room:02d}: "
                    guest_in_room = self._hotel[room]
                    while guest_in_room is not None:
                        hotel_string += f"{guest_in_room.get_name()}{self._NEXT_GUEST}"
                        guest_in_room = guest_in_room.get_next()
                    hotel_string += ""
        return hotel_string

In [22]:
test = HotelAlphabetical_SOLUTION(4)
# Obligatory statement about how bad idea is to write multi-statement lines
# but it is just for testing purposes here
test.add_guest("Alice")
test.add_guest("Aaron")
test.add_guest("Ava")
test.add_guest("Bob")
test.add_guest("Charlie")
test.add_guest("David")
test.add_guest("Eve")
test.add_guest("Zara")
test.add_guest("Yvonne")
test.add_guest("Xander")
test.add_guest("Mallory")
test.add_guest("Trent")
test.add_guest("Peggy")
test.add_guest("Victor")
test.add_guest("Walter")
print(test)


There are 30 guest(s) in your hotel.
The hotel has a capacity of 16 rooms. and is using 12 room(s).
The load factor is 0.75. The 30 guest(s) are:
	Room 00: Peggy --> 
	Room 01: Ava --> Aaron --> Alice --> 
	Room 02: Bob --> 
	Room 03: Charlie --> 
	Room 04: Trent --> David --> 
	Room 05: Eve --> 
	Room 06: Victor --> 
	Room 07: Walter --> 
	Room 08: Xander --> 
	Room 09: Yvonne --> 
	Room 10: Zara --> 
	Room 13: Mallory --> 
